# Exposing Parallelism in Loops
Loop transformations refer to changing the structure of the loops, so as to improve performance.

Transformations MUST be semantic-preserving, i.e., the loop must produce correct output/results after transformations. Make sure you do not voilate any dependence while modifying the loops.

## Important Tranformations
1. Loop Distribution
2. Loop Permutation/Interchange


## Loop Distribution
Here we decompose the loop(s) into several loops. This tranformation may allow us to parallelize at least one of the loops. Therefore, the overall performance of the program should improve. 

In [1]:
import numpy as np
import multiprocessing
import ctypes
from multiprocessing import Pool, Array
import math
import time
from PIL import Image

def checksum(array, N):
    checksum = 0
    for i in range(N):
        for j in range(N):
            checksum += array[i][j]
    print("Checksum: ", checksum)
    return

N = 2000
A = np.zeros((N,N))
B = np.zeros((N,N))
C = np.random.rand(N,N)
D = np.random.rand(N,N)

In [2]:
%%time
for i in range(1,N):
    for j in range(1,N):
        A[i][j] = C[i][j] + D[i][j]  # Statement 1
        B[i][j] = A[i-1][j-1]*2.0    # Statement 2: (i,j) dependent on (i-1,j-1) - Can't be parallelized

Wall time: 8.84 s


In [3]:
checksum(B,N) # check output

Checksum:  7980513.338547243


In [4]:
%%time
# But this can be distributed
for i in range(1,N):
    for j in range(1,N):
        A[i][j] = C[i][j] + D[i][j]  # Statement 1 - Can be parallelized
        
for i in range(1,N):
    for j in range(1,N):      
        B[i][j] = A[i-1][j-1]*2.0    # Statement 2: Can be parallelized

Wall time: 8.68 s


In [5]:
checksum(B,N) # check output again

Checksum:  7980513.338547243


In [6]:
shared_array_base_A = multiprocessing.Array(ctypes.c_float, N*N)
shared_A = np.ctypeslib.as_array(shared_array_base_A.get_obj())
shared_A = shared_A.reshape(N,N)

shared_array_base_B = multiprocessing.Array(ctypes.c_float, N*N)
shared_B = np.ctypeslib.as_array(shared_array_base_B.get_obj())
shared_B = shared_B.reshape(N,N)

shared_array_base_C = multiprocessing.Array(ctypes.c_float, N*N)
shared_C = np.ctypeslib.as_array(shared_array_base_C.get_obj())
shared_C = shared_C.reshape(N,N)

shared_array_base_D = multiprocessing.Array(ctypes.c_float, N*N)
shared_D = np.ctypeslib.as_array(shared_array_base_D.get_obj())
shared_D = shared_D.reshape(N,N)
np.copyto(shared_C,C)
np.copyto(shared_D,D)

In [7]:
def loop1(start,end):
    for i in range(start,end):
        for j in range(1,N):
            shared_A[i][j] = shared_C[i][j] + shared_D[i][j]
    return
            
def loop2(start,end):
    for i in range(start,end):
        for j in range(1,N):
            shared_B[i][j] = shared_A[i-1][j-1]*2.0
    return

In [8]:
%%time
p1 = multiprocessing.Process(target=loop1, args=(1,int(N/4),))
p1.start()
p2 = multiprocessing.Process(target=loop1, args=(int(N/4),int(N/2),))
p2.start()
p3 = multiprocessing.Process(target=loop1, args=(int(N/2),int((3*N)/4),))
p3.start()
p4 = multiprocessing.Process(target=loop1, args=(int((3*N)/4),N,))
p4.start()
p1.join()
p2.join()
p3.join()
p4.join()
p1 = multiprocessing.Process(target=loop2, args=(1,int(N/4),))
p1.start()
p2 = multiprocessing.Process(target=loop2, args=(int(N/4),int(N/2),))
p2.start()
p3 = multiprocessing.Process(target=loop2, args=(int(N/2),int((3*N)/4),))
p3.start()
p4 = multiprocessing.Process(target=loop2, args=(int((3*N)/4),N,))
p4.start()
p1.join()
p2.join()
p3.join()
p4.join()

Wall time: 342 ms


In [9]:
checksum(shared_A,N) # check output again

Checksum:  0.0


## Loop Permutation/Interchange
This transformation change the order of loops to get better parallelism.


In [10]:
C = np.random.rand(N,N)
D = np.random.rand(N,N)
A = np.copy(C) # Copy original C into A

In [11]:
%%time
for i in range(N-1):                   # Can't be parallelized
    for j in range(N-1):               # Can be parallelized 
        C[i+1][j] = C[i][j] * D[i][j]  # Statement 1: (i+1,j) dependent on (i,j)

Wall time: 5.06 s


In [12]:
checksum(C,N) # check output

Checksum:  3005.510857129583


In [13]:
C = np.copy(A) # Get original C values

In [14]:
%%time
for j in range(N-1):                   # Can be parallelized
    for i in range(N-1):               # Can't be parallelized 
        C[i+1][j] = C[i][j] * D[i][j]  # Statement 1: (i+1,j) dependent on (i,j)

Wall time: 5 s


In [15]:
checksum(C,N) # check output again

Checksum:  3005.510857129583


In [16]:
C = np.copy(A) # Get original C values
shared_array_base_C = multiprocessing.Array(ctypes.c_float, N*N)
shared_C = np.ctypeslib.as_array(shared_array_base_C.get_obj())
shared_C = shared_C.reshape(N,N)

shared_array_base_D = multiprocessing.Array(ctypes.c_float, N*N)
shared_D = np.ctypeslib.as_array(shared_array_base_D.get_obj())
shared_D = shared_D.reshape(N,N)
np.copyto(shared_C,C)
np.copyto(shared_D,D)

def parloop(start,end):
    for j in range(start,end):             # Can be parallelized
        for i in range(N-1):               # Can't be parallelized 
            shared_C[i+1][j] = shared_C[i][j] * shared_D[i][j]  # Statement 1: (i+1,j) dependent on (i,j)
    return

In [17]:
%%time
p1 = multiprocessing.Process(target=parloop, args=(0,int(N/4),))
p1.start()
p2 = multiprocessing.Process(target=parloop, args=(int(N/4),int(N/2),))
p2.start()
p3 = multiprocessing.Process(target=parloop, args=(int(N/2),int((3*N)/4),))
p3.start()
p4 = multiprocessing.Process(target=parloop, args=(int((3*N)/4),N-1,))
p4.start()
p1.join()
p2.join()
p3.join()
p4.join()

Wall time: 141 ms


In [18]:
checksum(shared_C,N) # check output again

Checksum:  1998826.0171461257


## Example: Sobel Image Filter (Edge Detection)

![title](peru.jpeg)

In [19]:
%%time
def sobelfilter(img, newimg, widht, height):
    for x in range(1, width-1):  # ignore the edge pixels for simplicity (1 to width-1)
        for y in range(1, height-1): # ignore edge pixels for simplicity (1 to height-1)

            # initialise Gx to 0 and Gy to 0 for every pixel
            Gx = 0
            Gy = 0
            # top left pixel
            p = img.getpixel((x-1, y-1))
            r = p[0]
            g = p[1]
            b = p[2]

            # intensity ranges from 0 to 765 (255 * 3)
            intensity = r + g + b

            # accumulate the value into Gx, and Gy
            Gx += -intensity
            Gy += -intensity

            # remaining left column
            p = img.getpixel((x-1, y))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += -2 * (r + g + b)

            p = img.getpixel((x-1, y+1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += -(r + g + b)
            Gy += (r + g + b)

            # middle pixels
            p = img.getpixel((x, y-1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gy += -2 * (r + g + b)

            p = img.getpixel((x, y+1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gy += 2 * (r + g + b)

            # right column
            p = img.getpixel((x+1, y-1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += (r + g + b)
            Gy += -(r + g + b)

            p = img.getpixel((x+1, y))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += 2 * (r + g + b)

            p = img.getpixel((x+1, y+1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += (r + g + b)
            Gy += (r + g + b)

            # calculate the length of the gradient (Pythagorean theorem)
            length = math.sqrt((Gx * Gx) + (Gy * Gy))

            # normalise the length of gradient to the range 0 to 255
            length = length / 4328 * 255

            length = int(length)

            # draw the length in the edge image
            #newpixel = img.putpixel((length,length,length))
            newimg.putpixel((x,y),(length,length,length))
    return

path = "peru.jpeg" # Your image path 
img = Image.open(path)
width, height = img.size
newimg = Image.new("RGB", (width,height), "white")
sobelfilter(img,newimg,width,height)
newimg.save("peru_new.jpeg")

FileNotFoundError: [Errno 2] No such file or directory: 'peru.jpeg'

![title](peru_new.jpeg)

### Parallel Sobel Filter

In [20]:
path = "peru.jpeg" # Your image path 
img1 = Image.open(path)
img2 = Image.open(path)
img3 = Image.open(path)
img4 = Image.open(path)
width, height = img1.size
newimg = Image.new("RGB", (width,height), "white")

shared_array_base = multiprocessing.Array(ctypes.c_int, width*height)
shared_array = np.ctypeslib.as_array(shared_array_base.get_obj())
shared_array = shared_array.reshape(width, height)

FileNotFoundError: [Errno 2] No such file or directory: 'peru.jpeg'

In [ ]:
def parallelsobelfilter(img, startwidth, endwidth, height, def_param=shared_array):
    for x in range(startwidth, endwidth):  
        for y in range(1, height-1): 
            # initialise Gx to 0 and Gy to 0 for every pixel
            Gx = 0
            Gy = 0

            # top left pixel
            p = img.getpixel((x-1, y-1))
            r = p[0]
            g = p[1]
            b = p[2]

            # intensity ranges from 0 to 765 (255 * 3)
            intensity = r + g + b

            # accumulate the value into Gx, and Gy
            Gx += -intensity
            Gy += -intensity

            # remaining left column
            p = img.getpixel((x-1, y))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += -2 * (r + g + b)

            p = img.getpixel((x-1, y+1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += -(r + g + b)
            Gy += (r + g + b)

            # middle pixels
            p = img.getpixel((x, y-1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gy += -2 * (r + g + b)

            p = img.getpixel((x, y+1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gy += 2 * (r + g + b)

            # right column
            p = img.getpixel((x+1, y-1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += (r + g + b)
            Gy += -(r + g + b)

            p = img.getpixel((x+1, y))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += 2 * (r + g + b)

            p = img.getpixel((x+1, y+1))
            r = p[0]
            g = p[1]
            b = p[2]

            Gx += (r + g + b)
            Gy += (r + g + b)

            # calculate the length of the gradient (Pythagorean theorem)
            length = math.sqrt((Gx * Gx) + (Gy * Gy))

            # normalise the length of gradient to the range 0 to 255
            length = length / 4328 * 255

            length = int(length)

            # draw the length in the edge image
            #newpixel = img.putpixel((length,length,length))
            shared_array[x][y] = length
    return



In [ ]:
%%time
p1 = multiprocessing.Process(target=parallelsobelfilter, args=(img1,1, int(width/4),height,))
p1.start()
p2 = multiprocessing.Process(target=parallelsobelfilter, args=(img2,int(width/4), int(width/2),height,))
p2.start()
p3 = multiprocessing.Process(target=parallelsobelfilter, args=(img3,int(width/2), int((3*width)/4),height,))
p3.start()
p4 = multiprocessing.Process(target=parallelsobelfilter, args=(img4,int((3*width)/4), width-1,height,))
p4.start()
p1.join()
p2.join()
p3.join()
p4.join()
for x in range(1, width-1): 
    for y in range(1, height-1):
        newimg.putpixel((x,y),(shared_array[x][y],shared_array[x][y],shared_array[x][y]))
newimg.save("peru_new.jpeg")

![title](peru_new.jpeg)